In [1]:
import pandas as pd

In [9]:
df_1 = pd.read_excel('finetune/data/question-cls/A 음식점(15,726)_new.xlsx')

In [10]:
df_1.head()

,SPEAKER,SENTENCE,DOMAINID,DOMAIN,CATEGORY,SPEAKERID,SENTENCEID,MAIN,SUB,QA,QACNCT,MQ,SQ,UA,SA,개체명,용어사전,지식베이스
0,고객,지금 배달되나요?,A,음식점,배달음식점,1,1,배달가능문의,NaN,Q,NaN,지금 배달되나요?,NaN,NaN,NaN,"수고, 지금, 배달",NaN,NaN
1,점원,아 네 배달됩니다,A,음식점,배달음식점,0,2,배달가능문의,NaN,A,NaN,NaN,NaN,NaN,아 네 배달됩니다,배달,NaN,NaN
2,고객,짬뽕류는 어떤 게 있나요? 잘 나가는 짬뽕 있나요?,A,음식점,배달음식점,1,3,베스트메뉴문의추천요청,NaN,Q,NaN,짬뽕류는 어떤 게 있나요? 잘 나가는 짬뽕 있나요?,NaN,NaN,NaN,짬뽕,NaN,짬뽕/메뉴
3,점원,특해물 짬뽕도 있고 전복 새우 짬뽕도 있고 해물 종류도 새우 홍합 전복 없는 게 없습니다,A,음식점,배달음식점,0,4,베스트메뉴문의추천요청,NaN,A,NaN,NaN,NaN,NaN,특해물 짬뽕도 있고 전복 새우 짬뽕도 있고 해물 종류도 새우 홍합 전복 없는 게 없습니다,"특해물 짬뽕, 전복 새우 짬뽕, 해물 종류, 새우, 홍합, 전복",NaN,"특해물 짬뽕/메뉴, 전복 새우 짬뽕/메뉴, 새우/해물/재료, 홍합/해물/재료, 전복..."
4,고객,전복 들어가는 거는 특해물 짬뽕 시켜야 돼요?,A,음식점,배달음식점,1,5,메뉴정보문의,NaN,Q,NaN,전복 들어가는 거는 특해물 짬뽕 시켜야 돼요?,NaN,NaN,NaN,"전복, 특해물 짬뽕",NaN,"전복/해물/재료, 특해물 짬뽕/메뉴"


In [11]:
len(df_1)

15726

In [12]:
# get df_1['MAIN'] contain word '문의' and df_1['MQ'] is not NaN
df_1_question = df_1[df_1['MAIN'].str.contains('문의') & df_1['MQ'].notnull()]['MQ']

In [15]:
len(df_1_question)

4218

In [16]:
df_1_question.head()

0                       지금 배달되나요?
2    짬뽕류는 어떤 게 있나요? 잘 나가는 짬뽕 있나요?
4       전복 들어가는 거는 특해물 짬뽕 시켜야 돼요?
6      전복 들어가고 여러 가지 또 딴 것도 들어가죠?
8            마차이 짬뽕밥은 돼지고기 들어가나요?
Name: MQ, dtype: object

In [17]:
df_1_not_question = df_1[df_1['SA'].notnull()]['SA']

In [18]:
len(df_1_not_question)

6037

In [19]:
df_1_not_question.head()

1                                            아 네 배달됩니다
3    특해물 짬뽕도 있고 전복 새우 짬뽕도 있고 해물 종류도 새우 홍합 전복 없는 게 없습니다
5                                   전복 짬뽕 시키면 전복이 들어가죠
7                                                   네네
9                                   짬뽕은 돼지고기 약간씩 들어갑니다
Name: SA, dtype: object

In [28]:
# for all excel file in data folder
import os

question_list = []
not_question_list = []


for file in os.listdir('data'):
    df = pd.read_excel('data/' + file)
    df_question = df[df['MAIN'].str.contains('문의') & df['MQ'].notnull()]['MQ']
    df_not_question = df[df['SA'].notnull()]['SA']
    question_list += (df_question.to_list())
    not_question_list += (df_not_question.to_list())


In [30]:
len(question_list)

33117

In [31]:
len(not_question_list)

33798

# TRAIN TEST SPLIT
---

In [37]:
#imoprt train_test_split
from sklearn.model_selection import train_test_split

question_train, question_test = train_test_split(question_list, test_size=0.2, random_state=42)
not_question_train, not_question_test = train_test_split(not_question_list, test_size=0.2, random_state=42)

In [40]:
print(len(question_train), len(question_test), len(not_question_train), len(not_question_test))

26493 6624 27038 6760


In [43]:
print(question_train[0])

선생님은 몇 분?


In [47]:
out_file_train_head = ["document\tlabel"]
out_file_train = []
for question in question_train:
    out_file_train.append(f"{question}\t1")
for not_question in not_question_train:
    out_file_train.append(f"{not_question}\t0")
import random
random.shuffle(out_file_train)
out_file_train = out_file_train_head + out_file_train
print(len(out_file_train))
#save out_file to txt file
with open('data/train.txt', 'w',encoding="utf-8") as f:
    for line in out_file_train:
        f.write(line + "\n")

53532


In [49]:
out_file_test_head = ["document\tlabel"]
out_file_test = []
for question in question_test:
    out_file_test.append(f"{question}\t1")
for not_question in not_question_test:
    out_file_test.append(f"{not_question}\t0")
import random
random.shuffle(out_file_test)
out_file_test = out_file_test_head + out_file_test
print(len(out_file_test))
#save out_file to txt file
with open('data/test.txt', 'w',encoding="utf-8") as f:
    for line in out_file_test:
        f.write(line + "\n")

13385


In [34]:
from transformers import ElectraForSequenceClassification, ElectraTokenizer